In [ ]:
import numpy as np,pandas as pd,matplotlib.pyplot as plt
from numba import jit
import warnings
warnings.filterwarnings("ignore")

# Forward Propagation 

In [ ]:
w1 = np.random.randint(1,100,(10,3))/100
b1 = (np.random.randint(1,100,10)/100).reshape(10,1)

w2 = np.random.randint(1,100,(10,10))/100
b2 = (np.random.randint(1,100,10)/100).reshape(10,1)

w3 = np.random.randint(1,100,(3,10))/100
b3 = (np.random.randint(1,100,3)/100).reshape(3,1)
      
# w = np.array([w1,w2])
# b = np.array([b1,b2])

In [ ]:
db = pd.read_csv("data")
db.drop("Unnamed: 0", axis=1,inplace=True)
x = np.array(db.iloc[:,:3])
x = x.reshape(100000,3,1)

In [ ]:
def weighted_inputs(w,x,b):
    z=[]
    for i in x:
        z += [np.dot(w,i) + b]
    return np.array(z)

In [ ]:
def sigmoid(z1):
    return 1/(1+np.exp(-z1))

def sigmoid_(z1):
    return sigmoid(z1)*(1-sigmoid(z1))

def relu(z1):
    return z1*(z1>=0)
    
def relu_(z1):
    return 1*(z1>=0)

def linear(z1):
    return z1

def linear_(z1):
    return np.ones(z1.shape)

In [ ]:
z1 = weighted_inputs(w1,x,b1)

a1 = relu(z1)

z2 = weighted_inputs(w2,a1,b2)

a2 = relu(z2)

z3 = weighted_inputs(w3,a2,b3)

a3 = linear(z3)

a0 = x

In [ ]:
relu(np.array([-1,1,-2]))

In [ ]:
a0.shape,relu_(z1).shape

# Back Propagation 

In [ ]:
target = np.array(db.iloc[:,-3:])
target = target.reshape(100000,3,1)

In [ ]:
def cost_fn(target,output):
    return (1/(2*len(target)))*np.sum(np.square(np.linalg.norm(target-a3)))

In [ ]:
lr = 0.3

In [ ]:
iterations = 50

In [ ]:
costs=[]

for epoch in range(iterations):
    e3 = (a3 - target) * linear_(z3)

    e2 = np.array([(w3.T @ e3[i]) for i in range(len(target))])
    e2 = e2 * relu_(z2)
    
    e1 = np.array([(w2.T @ e2[i]) for i in range(len(target))])
    e1 = e1 * relu_(z1)

    del_w1 = np.array([np.dot(a0[i],e1[i].T).T for i in range(len(target))])
    
    del_w2 = np.array([np.dot(a1[i],e2[i].T).T for i in range(len(target))])
    
    del_w3 = np.array([np.dot(a2[i],e3[i].T).T for i in range(len(target))])
    
    del_b1 = e1

    del_b2 = e2
    
    del_b3 = e3
    
    w1 = w1 - (lr/len(target)) * np.sum(del_w1,axis=0)
                                        
    w2 = w2 - (lr/len(target)) * np.sum(del_w2,axis=0)
    
    w3 = w3 - (lr/len(target)) * np.sum(del_w3,axis=0)

    b1 = b1 - (lr/len(w1)) * np.sum(del_b1, axis = 0)

    b2 = b2 - (lr/len(w2)) * np.sum(del_b2, axis = 0)
    
    b3 = b3 - (lr/len(w3)) * np.sum(del_b3, axis = 0)
    
    z1 = weighted_inputs(w1,x,b1)

    a1 = relu(z1)

    z2 = weighted_inputs(w2,a1,b2)

    a2 = relu(z2)

    z3 = weighted_inputs(w3,a2,b3)

    a3 = linear(z3)
    
    costs+=[cost_fn(target,a3)]
    print("Epoch {}".format(epoch+1))
    print("Cost: ",cost_fn(target,a3),end='\n\n')

In [ ]:
def predict(x,y,z):
    x = np.array([x,y,z].reshape(3,1))
    z1 = weighted_inputs(w1,x,b1)

    a1 = relu(z1)

    z2 = weighted_inputs(w2,a1,b2)

    a2 = relu(z2)

    z3 = weighted_inputs(w3,a2,b3)

    a3 = linear(z3)
    
    return a3
    

def get_array(x,y,z,lx,ly,lz):
    for i in range(iterations):
        x,y,z = predict(x,y,z)
        lx+=[x]
        ly+=[y]
        lz+=[z]
        
get_array(0.4,0.4,0.4,lx,ly,lz)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot(lx,ly,lz,lw=0.1)

Model is able to reproduce an estimate of the Lorenz Attractor